In [38]:
import pandas as pd

# Charger les données
df = pd.read_csv("full_grouped.csv")  

# Renommer les colonnes pour simplifier
df = df.rename(columns={
    'Date': 'Date',
    'Country/Region': 'Country',
    'Confirmed': 'Confirmed',
    'Deaths': 'Deaths',
    'Recovered': 'Recovered',
    'New cases': 'New cases',
    'New deaths': 'New deaths',
    'New recovered': 'New recovered'
})

# Convertir la date et créer une colonne mois
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()

# Garder uniquement les colonnes nécessaires
df = df[['Month', 'Country', 'Confirmed', 'Deaths', 'Recovered',
         'New cases', 'New deaths', 'New recovered']]

# Convertir les colonnes temporelles en valeurs absolues
for col in ['New cases', 'New deaths', 'New recovered']:
    df[col] = df[col].abs()

# Grouper par mois et pays
df_grouped = df.groupby(['Month', 'Country'], as_index=False).agg({
    'Confirmed': 'last',
    'Deaths': 'last',
    'Recovered': 'last',
    'New cases': 'sum',
    'New deaths': 'sum',
    'New recovered': 'sum'
})

# Ajouter une colonne Id auto-incrémentée
df_grouped.insert(0, 'Id', range(1, len(df_grouped) + 1))

# Ajouter une colonne 'nom' avec la valeur 'COVID-19' au début du DataFrame
df_grouped.insert(1, 'Name', 'COVID-19')

# Vérifier la cohérence des données
if df.isnull().any().any():
    print("Les données contiennent des valeurs nulles. Vérifiez le fichier source.")

# Vérifier que les colonnes numériques ne contiennent pas de valeurs négatives
for col in ['Confirmed', 'Deaths', 'Recovered', 'New cases', 'New deaths', 'New recovered']:
    if (df[col] < 0).any():
        print(f"La colonne '{col}' contient des valeurs négatives.")

# Vérifier que les colonnes nécessaires sont présentes
required_columns = ['Month', 'Country', 'Confirmed', 'Deaths', 'Recovered', 'New cases', 'New deaths', 'New recovered']
for col in required_columns:
    if col not in df.columns:
        print(f"La colonne requise '{col}' est manquante dans les données.")

MARGIN = 100  # Tolérance pour les incohérences
# Vérification des incohérences
# Initialiser une liste pour stocker les incohérences
inconsistencies = []

# Vérification des incohérences dans les données
# Parcourir chaque pays et vérifier les incohérences
for country, group in df_grouped.groupby('Country'):
    group = group.sort_values('Month').reset_index(drop=True)


    for i in range(1, len(group)):
        current = group.iloc[i]
        prev = group.iloc[i - 1]

        # Vérification décès
        expected_deaths = prev['Deaths'] + current['New deaths']
        if abs(current['Deaths'] - expected_deaths) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Deaths]: "
                  f"{current['Deaths']} ≠ {prev['Deaths']} + {current['New deaths']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Deaths',
                'Actual': current['Deaths'],
                'Expected': expected_deaths
            })

        # Vérification cas confirmés
        expected_confirmed = prev['Confirmed'] + current['New cases']
        if abs(current['Confirmed'] - expected_confirmed) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Confirmed]: "
                  f"{current['Confirmed']} ≠ {prev['Confirmed']} + {current['New cases']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Confirmed',
                'Actual': current['Confirmed'],
                'Expected': expected_confirmed
            })

        # Vérification récupérations
        expected_recovered = prev['Recovered'] + current['New recovered']
        if abs(current['Recovered'] - expected_recovered) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Recovered]: "
                  f"{current['Recovered']} ≠ {prev['Recovered']} + {current['New recovered']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Recovered',
                'Actual': current['Recovered'],
                'Expected': expected_recovered
            })


inconsistencies_df = pd.DataFrame(inconsistencies)
inconsistencies_df.head()

# Sauvegarder le résultat dans le répertoire courant
df_grouped.to_csv("data_etl_output.csv", index=False)

print("\nETL terminé. Fichier sauvegardé sous : data_etl_output.csv")

⚠️ Incohérence pour 'Algeria' au mois de 2020-03-01 00:00:00 [Recovered]: 46 ≠ 0 + 200 (tolérance ±100)
⚠️ Incohérence pour 'Benin' au mois de 2020-05-01 00:00:00 [Confirmed]: 232 ≠ 64 + 377 (tolérance ±100)
⚠️ Incohérence pour 'China' au mois de 2020-04-01 00:00:00 [Recovered]: 76951 ≠ 74645 + 4006 (tolérance ±100)
⚠️ Incohérence pour 'Cote d'Ivoire' au mois de 2020-06-01 00:00:00 [Recovered]: 4273 ≠ 1435 + 3460 (tolérance ±100)
⚠️ Incohérence pour 'Ecuador' au mois de 2020-05-01 00:00:00 [Confirmed]: 39098 ≠ 24934 + 17277 (tolérance ±100)
⚠️ Incohérence pour 'Finland' au mois de 2020-05-01 00:00:00 [Recovered]: 5500 ≠ 3000 + 2900 (tolérance ±100)
⚠️ Incohérence pour 'France' au mois de 2020-04-01 00:00:00 [Confirmed]: 167299 ≠ 52827 + 119170 (tolérance ±100)
⚠️ Incohérence pour 'France' au mois de 2020-05-01 00:00:00 [Deaths]: 28805 ≠ 24379 + 4860 (tolérance ±100)
⚠️ Incohérence pour 'France' au mois de 2020-05-01 00:00:00 [Confirmed]: 189009 ≠ 167299 + 22095 (tolérance ±100)
⚠️ Inco

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Charger les données
df = pd.read_csv("data_etl_output.csv")

# Créer des colonnes décalées pour les prédictions
for col in ['Confirmed', 'Deaths', 'Recovered']:
    df[f'{col}_lag1'] = df.groupby('Country')[col].shift(1)

# Supprimer les lignes avec des valeurs manquantes dues au décalage
df = df.dropna()

# Encoder les pays
le = LabelEncoder()
df['Country_encoded'] = le.fit_transform(df['Country'])

# Définir les features et la target
features = ['Confirmed_lag1', 'Deaths_lag1', 'Recovered_lag1', 'Country_encoded']
targets = ['Confirmed', 'Deaths', 'Recovered']

X = df[features]
y = df[targets]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


rfr_model = RandomForestRegressor(random_state=42)
rfr_model.fit(X_train, y_train)

rfr_rmse_confirmed = np.sqrt(mean_squared_error(y_test['Confirmed'], rfr_model.predict(X_test)[:, 0]))
rfr_rmse_deaths = np.sqrt(mean_squared_error(y_test['Deaths'], rfr_model.predict(X_test)[:, 1]))
rfr_rmse_recovered = np.sqrt(mean_squared_error(y_test['Recovered'], rfr_model.predict(X_test)[:, 2]))

print("Random Forest Regressor RMSE:")
print(f"Confirmed: {rfr_rmse_confirmed}, Deaths: {rfr_rmse_deaths}, Recovered: {rfr_rmse_recovered}")

# Gradient Boosting Regressor with MultiOutputRegressor
gbr_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
gbr_model.fit(X_train, y_train)

# Predictions with Gradient Boosting
gbr_y_pred = gbr_model.predict(X_test)

# Calculate RMSE for Gradient Boosting
gbr_rmse_confirmed = np.sqrt(mean_squared_error(y_test['Confirmed'], gbr_y_pred[:, 0]))
gbr_rmse_deaths = np.sqrt(mean_squared_error(y_test['Deaths'], gbr_y_pred[:, 1]))
gbr_rmse_recovered = np.sqrt(mean_squared_error(y_test['Recovered'], gbr_y_pred[:, 2]))

print("Gradient Boosting Regressor RMSE:")
print(f"Confirmed: {gbr_rmse_confirmed}, Deaths: {gbr_rmse_deaths}, Recovered: {gbr_rmse_recovered}")

# Linear Regression with MultiOutputRegressor
lr_model = MultiOutputRegressor(LinearRegression())
lr_model.fit(X_train, y_train)

# Predictions with Linear Regression
lr_y_pred = lr_model.predict(X_test)

# Calculate RMSE for Linear Regression
lr_rmse_confirmed = np.sqrt(mean_squared_error(y_test['Confirmed'], lr_y_pred[:, 0]))
lr_rmse_deaths = np.sqrt(mean_squared_error(y_test['Deaths'], lr_y_pred[:, 1]))
lr_rmse_recovered = np.sqrt(mean_squared_error(y_test['Recovered'], lr_y_pred[:, 2]))

print("Linear Regression RMSE:")
print(f"Confirmed: {lr_rmse_confirmed}, Deaths: {lr_rmse_deaths}, Recovered: {lr_rmse_recovered}")

Random Forest Regressor RMSE:
Confirmed: 56527.16753956689, Deaths: 3834.3655447739757, Recovered: 24347.824072305015
Gradient Boosting Regressor RMSE:
Confirmed: 80006.05882751447, Deaths: 2279.5740766256695, Recovered: 30815.436802704888
Linear Regression RMSE:
Confirmed: 55424.770449724325, Deaths: 2546.7357102811775, Recovered: 41015.71417962562


In [42]:
# Créer le pickle pour le modèle Random Forest
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(rfr_model, f)